In [1]:
import kaggle
import subprocess
import zipfile
import pandas as pd
import pandas as pd
import numpy as np

# Télécharger le dataset depuis Kaggle
subprocess.run(["kaggle", "datasets", "download", "-d", "retailrocket/ecommerce-dataset"])

# Décompresser le fichier zip téléchargé
with zipfile.ZipFile('ecommerce-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('../data/')

# Charger les fichiers CSV dans des DataFrames Pandas
category_tree = pd.read_csv('../data/category_tree.csv')
events = pd.read_csv('../data/events.csv')
item_properties_part1 = pd.read_csv('../data/item_properties_part1.csv')
item_properties_part2 = pd.read_csv('../data/item_properties_part2.csv')

# Afficher les premières lignes de chaque DataFrame pour inspection
print("Category Tree:")
print(category_tree.head())

print("\n Events:")
print(events.head())

print("\n Item Properties Part 1:")
print(item_properties_part1.head())

print("\n Item Properties Part 2:")
print(item_properties_part2.head())

Category Tree:
   categoryid  parentid
0        1016     213.0
1         809     169.0
2         570       9.0
3        1691     885.0
4         536    1691.0

 Events:
       timestamp  visitorid event  itemid  transactionid
0  1433221332117     257597  view  355908            NaN
1  1433224214164     992329  view  248676            NaN
2  1433221999827     111016  view  318965            NaN
3  1433221955914     483717  view  253185            NaN
4  1433221337106     951259  view  367447            NaN

 Item Properties Part 1:
       timestamp  itemid    property                            value
0  1435460400000  460429  categoryid                             1338
1  1441508400000  206783         888          1116713 960601 n277.200
2  1439089200000  395014         400  n552.000 639502 n720.000 424566
3  1431226800000   59481         790                       n15360.000
4  1431831600000  156781         917                           828513

 Item Properties Part 2:
       timestamp 